In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import torch 
from torch import nn

from PIL import Image
from pathlib import Path
import random
import matplotlib.pyplot as plt

import os

C:\Users\alexa\AppData\Local\Temp\ipykernel_27344\939756875.py:6: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

In [ ]:
import os
def walk_through_dir(dir_path):
  """
  Walks through dir_path returning its contents.
  Args:
    dir_path (str or pathlib.Path): target directory
  
  Returns:
    A print out of:
      number of subdiretories in dir_path
      number of images (files) in each subdirectory
      name of each subdirectory
  """
  for dirpath, dirnames, filenames in os.walk(dir_path):
    print(f"There are {len(dirnames)} directories and {len(filenames)} images in '{dirpath}'.")

https://www.kaggle.com/datasets/kmader/skin-cancer-mnist-ham10000

Original Challenge: https://challenge2018.isic-archive.com

https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/DBW86T
[1] Noel Codella, Veronica Rotemberg, Philipp Tschandl, M. Emre Celebi, Stephen Dusza, David Gutman, Brian Helba, Aadi Kalloo, Konstantinos Liopyris, Michael Marchetti, Harald Kittler, Allan Halpern: “Skin Lesion Analysis Toward Melanoma Detection 2018: A Challenge Hosted by the International Skin Imaging Collaboration (ISIC)”, 2018; https://arxiv.org/abs/1902.03368

[2] Tschandl, P., Rosendahl, C. & Kittler, H. The HAM10000 dataset, a large collection of multi-source dermatoscopic images of common pigmented skin lesions. Sci. Data 5, 180161 doi:10.1038/sdata.2018.161 (2018).

For good notebook: https://www.kaggle.com/code/sid321axn/step-wise-approach-cnn-model-77-0344-accuracy

In [ ]:
general_path = Path("/kaggle/input/skin-cancer-mnist-ham10000")
walk_through_dir(general_path)

In [ ]:
df = pd.read_csv(general_path / 'HAM10000_metadata.csv')
print(f"The shape of the metadata is {df.shape}")
df.head()

In [2]:
summary_dict = {
    'dx_distribution': df['dx'].value_counts(),
    'dx_type_distribution': df['dx_type'].value_counts(),
    'age_distribution': df['age'].describe(),
    'sex_distribution': df['sex'].value_counts(),
    'localization_distribution': df['localization'].value_counts()
}
summary_dict

IndexError: Cannot choose from an empty sequence

In [ ]:
# Let's make a dictionary for each image with its metadata, just for convenience:
image_metadata_dict = df.set_index('image_id').T.to_dict()

In [ ]:
# Now, we can always look up the corresponding information from an image_id:
image_id = 'ISIC_0027419'
metadata = image_metadata_dict.get(image_id)
metadata

From the competition, we know that "Cases include a representative collection of all important diagnostic categories in the realm of pigmented lesions: Actinic keratoses and intraepithelial carcinoma / Bowen's disease (akiec), basal cell carcinoma (bcc), benign keratosis-like lesions (solar lentigines / seborrheic keratoses and lichen-planus like keratoses, bkl), dermatofibroma (df), melanoma (mel), melanocytic nevi (nv) and vascular lesions (angiomas, angiokeratomas, pyogenic granulomas and hemorrhage, vasc)."

Let's plot some of the information so that it becomes a bit more clear.

# Let's first look at the different types of pigmented lesions and how many we have available to us in the dataset.
import matplotlib.pyplot as plt
import seaborn as sns

dx_data = summary_dict['dx_distribution']

plt.figure(figsize=(10, 6))
sns.barplot(x=dx_data.index, y=dx_data.values)
plt.title('Dx Distribution')
plt.xlabel('type of pigmented lesion')
plt.ylabel('frequency')
plt.show()

In [ ]:
dx_data = summary_dict['dx_type_distribution']

plt.figure(figsize=(10, 6))
sns.barplot(x=dx_data.index, y=dx_data.values)
plt.title('Dx Type Distribution')
plt.xlabel('How the lesion type got confirmed')
plt.ylabel('frequency')
plt.show()

In [ ]:

plt.figure(figsize=(10, 6))
sns.histplot(df['age'])
plt.title('Age Distribution')
plt.xlabel('Age')
plt.ylabel('Frequency')
plt.show()

In [ ]:
dx_data = summary_dict['dx_distribution']

plt.figure(figsize=(10, 6))
sns.barplot(x=dx_data.index, y=dx_data.values)
plt.title('Dx Distribution')
plt.xlabel('type of pigmented lesion')
plt.ylabel('frequency')
plt.show()

In [ ]:
image_path_part_1 = general_path / Path("HAM10000_images_part_1")
path_list_part_1 = list(image_path_part_1.glob('*.jpg'))
len(path_list_part_1)

image_path_part_2 = general_path / Path("HAM10000_images_part_2")
path_list_part_2 = list(image_path_part_2.glob('*.jpg'))
len(path_list_part_2)

all_images_path = path_list_part_1 + path_list_part_2 
len(all_images_path)

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms

In [ ]:
# Write a transform for image to tensors
data_transform = transforms.Compose([
    transforms.Resize(size=(64, 64)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.ToTensor()
])

In [ ]:
image_path_part_1 = general_path / Path("HAM10000_images_part_1")
path_list_part_1 = list(image_path_part_1.glob('*.jpg'))
len(path_list_part_1)

In [ ]:
image_path_part_2 = general_path / Path("HAM10000_images_part_2")
path_list_part_2 = list(image_path_part_2.glob('*.jpg'))
len(path_list_part_2)

In [ ]:
all_images_path = path_list_part_1 + path_list_part_2 
len(all_images_path)

In [ ]:
def plot_transformed_images(image_paths, transform, n=3, seed=42):
    """Plots a series of random images from image_paths.

    Will open n image paths from image_paths, transform them
    with transform and plot them side by side.

    Args:
        image_paths (list): List of target image paths. 
        transform (PyTorch Transforms): Transforms to apply to images.
        n (int, optional): Number of images to plot. Defaults to 3.
        seed (int, optional): Random seed for the random generator. Defaults to 42.
    """
    random.seed(seed)
    random_image_paths = random.sample(image_paths, k=n)
    for image_path in random_image_paths:
        with Image.open(image_path) as f:
            fig, ax = plt.subplots(1, 2)
            ax[0].imshow(f) 
            ax[0].set_title(f"Original \nSize: {f.size}")
            ax[0].axis("off")

            # Transform and plot image
            # Note: permute() will change shape of image to suit matplotlib 
            # (PyTorch default is [C, H, W] but Matplotlib is [H, W, C])
            transformed_image = transform(f).permute(1, 2, 0) 
            ax[1].imshow(transformed_image) 
            ax[1].set_title(f"Transformed \nSize: {transformed_image.shape}")
            ax[1].axis("off")

            fig.suptitle(f"Class: {image_path.parent.stem}", fontsize=16)

plot_transformed_images(all_images_path, 
                        transform=data_transform, 
                        n=3)

In [ ]:
from sklearn.model_selection import train_test_split

train_paths, test_paths = train_test_split(all_images_path, test_size=0.20, shuffle=True, random_state=42)

In [ ]:
print(f"Number of train_paths: {len(train_paths)} and number of test_paths: {len(test_paths)}")

In [ ]:
general_path    

In [ ]:
import os
import shutil

base_path = '/kaggle/working'

train_dir = os.path.join(base_path, 'train_folder')
test_dir = os.path.join(base_path, 'test_folder')

if not os.path.exists(train_dir):
    os.makedirs(train_dir)
    
if not os.path.exists(test_dir):
    os.makedirs(test_dir)

def copy_images(image_paths, target_dir):
    for path in image_paths:
        
        filename = os.path.basename(path)
        new_path = os.path.join(target_dir, filename)
        
        shutil.copy(path, new_path)

if not os.listdir(train_dir):
    copy_images(train_paths, train_dir)
    
if not os.listdir(test_dir):
    copy_images(test_paths, test_dir)

In [ ]:
walk_through_dir('/kaggle/working')

In [ ]:
lesion_types = df['dx'].unique()

for lesion_type in lesion_types:
    os.makedirs(os.path.join(train_dir, lesion_type), exist_ok=True)
    os.makedirs(os.path.join(test_dir, lesion_type), exist_ok=True)
    
train_df, test_df = train_test_split(df, test_size=0.20, stratify=df['dx'], random_state=42)

def copy_images_to_subdir(df, base_dir, images_dir1, images_dir2):
    for index, row in df.iterrows():
        file_name = row['image_id'] + '.jpg'
        lesion_type = row['dx']
        
        source_path_1 = os.path.join(images_dir1, file_name)
        source_path_2 = os.path.join(images_dir2, file_name)
        
        if os.path.exists(source_path_1):
            source_path = source_path_1
        elif os.path.exists(source_path_2):
            source_path = source_path_2
        else:
            print(f"Image {file_name} not found in both directories.")
            continue  # Skip this file
        
        target_subdir = os.path.join(base_dir, lesion_type)
        shutil.copy(source_path, target_subdir)
        
images_part1 = general_path / 'HAM10000_images_part_1'
images_part2 = general_path / 'HAM10000_images_part_2'

copy_images_to_subdir(train_df, train_dir, images_part1, images_part2)
copy_images_to_subdir(test_df, test_dir, images_part1, images_part2)

In [ ]:
from torchvision import datasets

train_data = datasets.ImageFolder(root=train_dir,
                                  transform=data_transform,
                                  target_transform=None)

test_data = datasets.ImageFolder(root=test_dir,
                                 transform=data_transform)

print(f"Train data:\n{train_data}\nTest data:\n{test_data}")

In [ ]:
class_names = train_data.classes
class_dict = train_data.class_to_idx
class_names, class_dict

In [ ]:
def set_seeds(seed: int=42):
    """Sets random sets for torch operations.

    Args:
        seed (int, optional): Random seed to set. Defaults to 42.
    """
    # Set the seed for general torch operations
    torch.manual_seed(seed)
    # Set the seed for CUDA torch operations (ones that happen on the GPU)
    torch.cuda.manual_seed(seed)

There are very good pretrained models out there that have been extensively trained and tested. Some of these have been trained on ImageNet, and therefore we need to apply normalization to our images so that the models perform better.

from torch.utils.data import Subset

NUM_WORKERS = os.cpu_count()
BATCH_SIZE = 32

def create_dataloaders(
    train_dir: str, 
    test_dir: str, 
    transform: transforms.Compose, 
    batch_size: int, 
    fraction: float=1.0,
    num_workers: int=NUM_WORKERS
):
    """Creates training and testing DataLoaders.

    Takes in a training directory and testing directory path and turns
    them into PyTorch Datasets and then into PyTorch DataLoaders.

    Args:
    train_dir: Path to training directory.
    test_dir: Path to testing directory.
    transform: torchvision transforms to perform on training and testing data.
    batch_size: Number of samples per batch in each of the DataLoaders.
    num_workers: An integer for number of workers per DataLoader.

    Returns:
    A tuple of (train_dataloader, test_dataloader, class_names).
    Where class_names is a list of the target classes.
    Example usage:
        train_dataloader, test_dataloader, class_names = \
        = create_dataloaders(train_dir=path/to/train_dir,
                                test_dir=path/to/test_dir,
                                transform=some_transform,
                                batch_size=32,
                                num_workers=4)
    """
    # Use ImageFolder to create dataset(s)
    train_data = datasets.ImageFolder(train_dir, transform=transform)
    test_data = datasets.ImageFolder(test_dir, transform=transform)

    # Get class names
    class_names = train_data.classes
    
    # Generate a random subset of indices for train and test sets
    subset_indices_train = np.random.choice(len(train_data), int(len(train_data) * fraction), replace=False)
    subset_indices_test = np.random.choice(len(test_data), int(len(test_data) * fraction), replace=False)

    train_subset = Subset(train_data, subset_indices_train)
    test_subset = Subset(test_data, subset_indices_test)
    
    # Turn images into data loaders
    train_dataloader = DataLoader(
        train_subset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=num_workers,
        pin_memory=True,
    )
    test_dataloader = DataLoader(
        test_subset,
        batch_size=batch_size,
        shuffle=False, # don't need to shuffle test data
        num_workers=num_workers,
        pin_memory=True,
    )

    return train_dataloader, test_dataloader, class_names

In [ ]:
# These are the transforms we will use as they have been used for training the model
import torchvision
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT

automatic_transforms = weights.transforms()
print(f"Automatically created transforms: {automatic_transforms}")

In [ ]:
# Let's first start by training the model on 5 percent and 10 percent of the data
train_dataloader_5_percent, test_dataloader_5_percent, class_names = create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=automatic_transforms, # use automatic created transforms
    fraction=0.05,
    batch_size=BATCH_SIZE
)

train_dataloader_10_percent, test_dataloader_10_percent, class_names = create_dataloaders(
    train_dir=train_dir,
    test_dir=test_dir,
    transform=automatic_transforms, # use automatic created transforms
    fraction=0.1,
    batch_size=BATCH_SIZE
)

# Find the number of samples/batches per dataloader (using the same test_dataloader for both experiments)
print(f"Number of batches of size {BATCH_SIZE} in 5 percent training data: {len(train_dataloader_5_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in 10 percent training data: {len(train_dataloader_10_percent)}")
print(f"Number of batches of size {BATCH_SIZE} in testing data: {len(train_dataloader_5_percent)} (all experiments will use the same test set)")
print(f"Number of classes: {len(class_names)}, class names: {class_names}")

In [ ]:
# Let's have functions for creating the two models, as we will be experimenting with different hyperparameters.
OUT_FEATURES = len(class_names)

def create_effnetb0():
    weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT
    model = torchvision.models.efficientnet_b0(weights=weights).to(device)

    # 2. Freeze the base model layers
    for param in model.features.parameters():
        param.requires_grad = False

    # 3. Set the seeds
    set_seeds()

    # 4. Change the classifier head
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.2),
        nn.Linear(in_features=1280, out_features=OUT_FEATURES)
    ).to(device)

    # 5. Give the model a name
    model.name = "effnetb0"
    print(f"[INFO] Created new {model.name} model.")
    return model

def create_effnetb2():
    # 1. Get the base model with pretrained weights and send to target device
    weights = torchvision.models.EfficientNet_B2_Weights.DEFAULT
    model = torchvision.models.efficientnet_b2(weights=weights).to(device)

    # 2. Freeze the base model layers
    for param in model.features.parameters():
        param.requires_grad = False

    # 3. Set the seeds
    set_seeds()

    # 4. Change the classifier head
    model.classifier = nn.Sequential(
        nn.Dropout(p=0.3),
        nn.Linear(in_features=1408, out_features=OUT_FEATURES)
    ).to(device)

    # 5. Give the model a name
    model.name = "effnetb2"
    print(f"[INFO] Created new {model.name} model.")
    return model